**This notebook is an exercise in the [Natural Language Processing](https://www.kaggle.com/learn/natural-language-processing) course.  You can reference the tutorial at [this link](https://www.kaggle.com/matleonard/text-classification).**

---


# Natural Language Classification

You did such a great job for DeFalco's restaurant in the previous exercise that the chef has hired you for a new project.

The restaurant's menu includes an email address where visitors can give feedback about their food. 

The manager wants you to create a tool that automatically sends him all the negative reviews so he can fix them, while automatically sending all the positive reviews to the owner, so the manager can ask for a raise. 

You will first build a model to distinguish positive reviews from negative reviews using Yelp reviews because these reviews include a rating with each review. Your data consists of the text body of each review along with the star rating. Ratings with 1-2 stars count as "negative", and ratings with 4-5 stars are "positive". Ratings with 3 stars are "neutral" and have been dropped from the data.

Let's get started. First, run the next code cell.

In [1]:
import pandas as pd

# Set up code checking
from learntools.core import binder
binder.bind(globals())
from learntools.nlp.ex2 import *
print("\nSetup complete")


Setup complete


# Step 1: Evaluate the Approach

Is there anything about this approach that concerns you? After you've thought about it, run the function below to see one point of view.

In [2]:
# Check your answer (Run this code cell to receive credit!)
step_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> Any way of setting up an ML problem will have multiple strengths and weaknesses.  So you may have thought of different issues than listed here.

The strength of this approach is that it allows you to distinguish positive email messages from negative emails even though you don't have historical emails that you have labeled as positive or negative.

The weakness of this approach is that emails may be systematically different from Yelp reviews in ways that make your model less accurate. For example, customers might generally use different words or slang in emails, and the model based on Yelp reviews won't have seen these words.

If you wanted to see how serious this issue is, you could compare word frequencies between the two sources. In practice, manually reading a few emails from each source may be enough to see if it's a serious issue. 

If you wanted to do something fancier, you could create a dataset that contains both Yelp reviews and emails and see whether a model can tell a reviews source from the text content. Ideally, you'd like to find that model didn't perform well, because it would mean your data sources are similar. That approach seems unnecessarily complex here.

# Step 2: Review Data and Create the model

Moving forward with your plan, you'll need to load the data. Here's some basic code to load data and split it into a training and validation set. Run this code.

In [3]:
def load_data(csv_file, split=0.9):
    data = pd.read_csv(csv_file)
    
    # Shuffle data
    train_data = data.sample(frac=1, random_state=7)
    
    texts = train_data.text.values
    labels = [{"POSITIVE": bool(y), "NEGATIVE": not bool(y)}
              for y in train_data.sentiment.values]
    split = int(len(train_data) * split)
    
    train_labels = [{"cats": labels} for labels in labels[:split]]
    val_labels = [{"cats": labels} for labels in labels[split:]]
    
    return texts[:split], train_labels, texts[split:], val_labels

train_texts, train_labels, val_texts, val_labels = load_data('../input/nlp-course/yelp_ratings.csv')

You will use this training data to build a model. The code to build the model is the same as what you saw in the tutorial. So that is copied below for you.

First, run the cell below to look at a couple elements from your training data.

In [4]:
print('Texts from training data\n------')
print(train_texts[:2])
print('\nLabels from training data\n------')
print(train_labels[:2])


Texts from training data
------
["Some of the best sushi I've ever had....and I come from the East Coast.  Unreal toro, have some of it's available."
 "One of the best burgers I've ever had and very well priced. I got the tortilla burger and is was delicious especially with there tortilla soup!"]

Labels from training data
------
[{'cats': {'POSITIVE': True, 'NEGATIVE': False}}, {'cats': {'POSITIVE': True, 'NEGATIVE': False}}]


But because your data is different, there are **two lines in the modeling code cell that you'll need to change.** Can you figure out what they are? 

If you're not sure, take a second look at the data, and pay particular attention to the labels that should be fed to the text classifier.

In [5]:
import spacy

# Create an empty model
nlp = spacy.blank('en')

# Add the TextCategorizer to the empty model
textcat = nlp.add_pipe('textcat')

# Add labels to text classifier
textcat.add_label("NEGATIVE")
textcat.add_label("POSITIVE")

# Check your answer
step_2.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
# Lines below will give you a hint or solution code
#step_2.hint()
#step_2.solution()

# Step 3: Train Function

Implement a function `train` that updates a model with training data. Most of this is general data munging, which we've filled in for you. 

Just add the one line of code necessary to update your model.

In [7]:
import random
from spacy.util import minibatch
from spacy.training.example import Example

def train(model, train_data, optimizer, batch_size=8):
    losses = {}
    random.seed(1)
    random.shuffle(train_data)
    
    # train_data is a list of tuples [(text0, label0), (text1, label1), ...]
    for batch in minibatch(train_data, size=batch_size):
        # Split batch into text and labels
        for text, labels in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, labels)
            # TODO: Update model with texts and labels
            model.update([example], sgd=optimizer, losses=losses)
        
    return losses

# Check your answer
step_3.check()

[2021-11-24 14:43:38,287] [INFO] Created vocabulary
[2021-11-24 14:43:38,289] [INFO] Finished initializing nlp object
[2021-11-24 14:43:58,544] [INFO] Created vocabulary
[2021-11-24 14:43:58,547] [INFO] Finished initializing nlp object


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [8]:
# Lines below will give you a hint or solution code
#step_3.hint()
step_3.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

def train(model, train_data, optimizer, batch_size=8):
    losses = {}
    random.seed(1)
    random.shuffle(train_data)
    
    for batch in minibatch(train_data, size=batch_size):
        for text, labels in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, labels)
            # Update model with texts and labels
            model.update([example], sgd=optimizer, losses=losses)
        
    return losses

```

In [9]:
# Fix seed for reproducibility
spacy.util.fix_random_seed(1)
random.seed(1)

# This may take a while to run!
optimizer = nlp.begin_training()
train_data = list(zip(train_texts, train_labels))
losses = train(nlp, train_data, optimizer)
print(losses['textcat'])

[2021-11-24 14:44:18,490] [INFO] Created vocabulary
[2021-11-24 14:44:18,492] [INFO] Finished initializing nlp object


6553.196886404312


We can try this slightly trained model on some example text and look at the probabilities assigned to each label.

In [10]:
text = "This tea cup was full of holes. Do not recommend."
doc = nlp(text)
print(doc.cats)

{'NEGATIVE': 0.9697662591934204, 'POSITIVE': 0.030233776196837425}


These probabilities look reasonable. Now you should turn them into an actual prediction.

# Step 4: Making Predictions

Implement a function `predict` that predicts the sentiment of text examples. 
- First, tokenize the texts using `nlp.tokenizer()`. 
- Then, pass those docs to the TextCategorizer which you can get from `nlp.get_pipe()`. 
- Use the `textcat.predict()` method to get scores for each document, then choose the class with the highest score (probability) as the predicted class.

In [11]:
def predict(nlp, texts): 
    # Use the model's tokenizer to tokenize each input text
    docs = [nlp.tokenizer(text) for text in texts]
    
    # Use textcat to get the scores for each doc
    textcat = nlp.get_pipe("textcat")
    scores = textcat.predict(docs)
    
    # From the scores, find the class with the highest score/probability
    predicted_class = scores.argmax(axis=1)
    
    return predicted_class

# Check your answer
step_4.check()

[2021-11-24 14:58:32,490] [INFO] Created vocabulary
[2021-11-24 14:58:32,492] [INFO] Finished initializing nlp object


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [12]:
# Lines below will give you a hint or solution code
#step_4.hint()
step_4.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

def predict(nlp, texts): 
    # Use the model's tokenizer to tokenize each input text
    docs = [nlp.tokenizer(text) for text in texts]
    
    # Use textcat to get the scores for each doc
    textcat = nlp.get_pipe("textcat")
    scores = textcat.predict(docs)
    
    # From the scores, find the class with the highest score/probability
    predicted_class = scores.argmax(axis=1)
    
    return predicted_class
```

In [13]:
texts = val_texts[34:38]
predictions = predict(nlp, texts)

for p, t in zip(predictions, texts):
    print(f"{textcat.labels[p]}: {t} \n")

POSITIVE: Came over and had their "Pick 2" lunch combo and chose their best selling 1/2 chicken sandwich with quinoa.  Both were tasty, the chicken salad is a bit creamy but was perfect with quinoa on the side.  This is a good lunch joint, casual and clean! 

POSITIVE: Went here last night and got oysters, fried okra, fries, and onion rings. I cannot complain. The portions were great and tasty!!! I will definitely be back for more. I cannot wait to try the crawfish boudin and soft shell crab. 

POSITIVE: This restaurant was fantastic! 
The concept of eating without vision was intriguing. The dinner was filled with laughs and good conversation. 

We were lead in a line to our table and each person to their seat. This was not just dark but you could not see something right in front of your face. 

The waiters/waitresses were all blind and allowed us to see how aware you need to be without the vision. 

Taking away one sense is said to increase your other senses so as taste and hearing wh

It looks like your model is working well after going through the data just once. However you need to calculate some metric for the model's performance on the hold-out validation data.

# Step 5: Evaluate The Model

Implement a function that evaluates a `TextCategorizer` model. This function `evaluate` takes a model along with texts and labels. It returns the accuracy of the model, which is the number of correct predictions divided by all predictions.

First, use the `predict` method you wrote earlier to get the predicted class for each text in `texts`. Then, find where the predicted labels match the true "gold-standard" labels and calculate the accuracy.

In [14]:
def evaluate(model, texts, labels):
    """ Returns the accuracy of a TextCategorizer model. 
    
        Arguments
        ---------
        model: ScaPy model with a TextCategorizer
        texts: Text samples, from load_data function
        labels: True labels, from load_data function
    
    """
    # Get predictions from textcat model (using your predict method)
    predicted_class = predict(model, texts)
    
    # From labels, get the true class as a list of integers (POSITIVE -> 1, NEGATIVE -> 0)
    true_class = [int(each['cats']['POSITIVE']) for each in labels]
    
    # A boolean or int array indicating correct predictions
    correct_predictions = predicted_class == true_class
    
    # The accuracy, number of correct predictions divided by all predictions
    accuracy = correct_predictions.mean()
    
    return accuracy

# Check your answer
step_5.check()

[2021-11-24 14:58:52,464] [INFO] Created vocabulary
[2021-11-24 14:58:52,466] [INFO] Finished initializing nlp object


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [15]:
# Lines below will give you a hint or solution code
#step_5.hint()
step_5.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

    def evaluate(model, texts, labels):
        # Get predictions from textcat model
        predicted_class = predict(model, texts)
        
        # From labels, get the true class as a list of integers (POSITIVE -> 1, NEGATIVE -> 0)
        true_class = [int(each['cats']['POSITIVE']) for each in labels]
        
        # A boolean or int array indicating correct predictions
        correct_predictions = predicted_class == true_class
        
        # The accuracy, number of correct predictions divided by all predictions
        accuracy = correct_predictions.mean()
        
        return accuracy
    
```

In [16]:
accuracy = evaluate(nlp, val_texts, val_labels)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9149


With the functions implemented, you can train and evaluate in a loop.

In [17]:
# This may take a while to run!
n_iters = 5
for i in range(n_iters):
    losses = train(nlp, train_data, optimizer)
    accuracy = evaluate(nlp, val_texts, val_labels)
    print(f"Loss: {losses['textcat']:.3f} \t Accuracy: {accuracy:.3f}")

Loss: 4775.517 	 Accuracy: 0.924
Loss: 4499.027 	 Accuracy: 0.927
Loss: 4401.864 	 Accuracy: 0.933
Loss: 4358.448 	 Accuracy: 0.932
Loss: 4192.791 	 Accuracy: 0.923


# Step 6: Keep Improving

You've built the necessary components to train a text classifier with spaCy. What could you do further to optimize the model?

Run the next line to check your answer.

In [18]:
# Check your answer (Run this code cell to receive credit!)
step_6.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> Answer: There are various hyperparameters to work with here. The biggest one is the TextCategorizer architecture. You used the simplest model which trains faster but likely has worse performance than the CNN and ensemble models. 

## Keep Going

The next step is a big one. See how you can **[represent tokens as vectors that describe their meaning](https://www.kaggle.com/matleonard/word-vectors)**, and plug those into your machine learning models.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/natural-language-processing/discussion) to chat with other learners.*